In [23]:
import pandas as pd
import os
import re
import json

In [8]:
adjusted_forecast = pd.read_excel("C:/Users/rundx/idaes-gtep/gtep/data/123_Bus_Coal/ERCOT-Adjusted-Forecast.xlsb")

In [50]:
def load_solution():
    with open("C:/Users/rundx/idaes-gtep/First decent run 4_23_long/dispatchable_investments.json", "r") as fil:
        dispatchable_dict = json.load(fil)
    print(dispatchable_dict)
    dispatchable_dict = {k:v for k,v in dispatchable_dict.items() if "investmentStage[3]" in k and "gen" in k}
    print(dispatchable_dict)
    with open("C:/Users/rundx/idaes-gtep/First decent run 4_23_long/renewable_investments.json", "r") as fil:
        renewable_dict = json.load(fil)
    print(renewable_dict)
    renewable_dict = {k:v for k,v in renewable_dict.items() if "investmentStage[3]" in k}
    print(renewable_dict)
    dispatchable_list = [k.split(".")[-1] for k in dispatchable_dict.keys() if "Operational" in k or "Extended" in k or "Installed" in k or "Retired" in k]
    renewable_dict = {k.split(".")[-1]: v for k,v in renewable_dict.items()}
    combined_renewables = {}
    for k, v in renewable_dict.items():
        if combined_renewables.get(k):
            combined_renewables[k] += v
        else:
            combined_renewables[k] = v
    dispatchable_list += [k for k in renewable_dict]
    return dispatchable_list, combined_renewables
invested_gen_list, invested_renewable_dict = load_solution()

{'investmentStage[1].genOperational.1': True, 'investmentStage[1].genOperational.2': True, 'investmentStage[1].genOperational.3': True, 'investmentStage[1].genOperational.4': True, 'investmentStage[1].genOperational.5': True, 'investmentStage[1].genOperational.6': True, 'investmentStage[1].genOperational.7': True, 'investmentStage[1].genOperational.8': True, 'investmentStage[1].genOperational.10': True, 'investmentStage[1].genOperational.11': True, 'investmentStage[1].genOperational.12': True, 'investmentStage[1].genOperational.13': True, 'investmentStage[1].genOperational.14': True, 'investmentStage[1].genOperational.15': True, 'investmentStage[1].genOperational.16': True, 'investmentStage[1].genOperational.18': True, 'investmentStage[1].genOperational.25': True, 'investmentStage[1].genOperational.26': True, 'investmentStage[1].genOperational.27': True, 'investmentStage[1].genOperational.28': True, 'investmentStage[1].genOperational.44': True, 'investmentStage[1].genOperational.45': T

In [51]:
print(invested_gen_list)
print(invested_renewable_dict)

['1', '2', '3', '4', '5', '7', '8', '10', '11', '12', '13', '14', '15', '16', '18', '25', '27', '28', '44', '45', '47', '48', '49', '50', '51', '54', '60', '61', '62', '66', '70', '71', '72', '73', '75', '76', '85', '93', '94', '98', '100', '105', '107', '108', '112', '114', '118', '121', '122', '124', '129', '130', '137', '138', '144', '147', '148', '149', '150', '151', '155', '156', '166', '171', '174', '176', '178', '179', '180', '182', '183', '184', '186', '189', '193', '195', '197', '201', '202', '203', '207', '208', '209', '211', '212', '213', '215', '216', '220', '222', '223', '224', '226', '227', '228', '229', '230', '231', '234', '235', '239', '242', '243', '244', '246', '253', '256', '257', '259', '262', '263', '264', '268', '271', '280', 'ct_fe_1_2-c', 'ct_fe_1_3-c', 'ct_fe_8_1-c', 'ct_fe_8_2-c', 'ct_fe_8_3-c', 'ct_fe_15_1-c', 'ct_fe_15_3-c', 'ct_fe_17_1-c', 'ct_fe_17_2-c', 'ct_fe_17_3-c', 'ct_fe_18_1-c', 'ct_fe_18_2-c', 'ct_fe_18_3-c', 'ct_fe_19_1-c', 'ct_fe_19_2-c', 'ct_fe

In [ ]:
renewable_df = pd.Series(invested_renewable_dict)
renewable_df

19     352.200855
20     352.200855
23     108.800000
24     192.500000
31     125.000000
          ...    
273     30.000000
276      9.000000
277     11.000000
278     18.000000
282    362.000000
Length: 150, dtype: float64

In [57]:
input_directory = "C:/Users/rundx/idaes-gtep/gtep/data/123_Bus_Coal"
output_directory = "C:/Users/rundx/idaes-gtep/gtep/data/123_Bus_Coal/Prescient"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
def populate_generators(data_input_path, data_output_path):
    gen_df = pd.read_csv(data_input_path + "/gen.csv")
    columns = gen_df.columns.to_list()
    columns.remove('Output_pct_2')
    columns.remove('Output_pct_3')
    columns.remove('Output_pct_4')
    columns.remove('HR_incr_2')
    columns.remove('HR_incr_3')
    columns.remove('HR_incr_4')
    gen_df["Fuel Price $/MMBTU"] = 1
    gen_df["Output_pct_0"]= 0
    gen_df["HR_avg_0"].fillna(0, inplace=True)
    gen_df["HR_incr_1"] = gen_df["fuel_cost3"]
    gen_df = gen_df[gen_df["GEN UID"].isin(invested_gen_list)]
    gen_df["PMax MW"].update(renewable_df)
    print(gen_df)
    gen_df.to_csv(os.path.join(data_output_path, "gen.csv"), index=False)
    
populate_generators(input_directory, output_directory)

           GEN UID  Bus ID Unit Type Fuel  MW Inj  MVAR Inj  V Setpoint p.u.  \
0                1     107       NUC    N     NaN       NaN                1   
1                2     100        CT    G     NaN       NaN                1   
2                3     100        CT    G     NaN       NaN                1   
3                4     100        CT    G     NaN       NaN                1   
4                5     100        CT    G     NaN       NaN                1   
..             ...     ...       ...  ...     ...       ...              ...   
401  ct_fe_115_2-c     115        CT    G     NaN       NaN                1   
402  ct_fe_115_3-c     115        CT    G     NaN       NaN                1   
403  ct_fe_118_1-c     118        CT    G     NaN       NaN                1   
404  ct_fe_118_2-c     118        CT    G     NaN       NaN                1   
405  ct_fe_118_3-c     118        CT    G     NaN       NaN                1   

     PMax MW  PMin MW  QMax MVAR  ...  

In [58]:
def filter_pointers(data_input_path, data_output_path):
    input_pointers_df = pd.read_csv(
        os.path.join(data_input_path, "timeseries_pointers.csv")
    )
    output_generators_df = pd.read_csv(os.path.join(data_output_path, "gen.csv"))
    matching_gen_list = [gen for gen in output_generators_df["GEN UID"]]
    output_df = input_pointers_df[
        input_pointers_df["Object"].isin(matching_gen_list)
        | input_pointers_df["Category"]
        != "Generator"
    ]
    output_df.to_csv(os.path.join(data_output_path, "timeseries_pointers.csv"))
filter_pointers(input_directory, output_directory)

In [66]:
import shutil
def clone_timeseries(data_input_path, data_output_path):

    if not os.path.exists(data_output_path):
        os.makedirs(data_output_path)

    file_list = os.listdir(data_input_path,)
    file_list.remove("timeseries_pointers.csv")
    file_list.remove("gen.csv")
    file_list.remove("branch.csv")
    file_list.remove("Prescient")
    for file in file_list:
        print(file)
        if not "csv" in file:
            print(file)
            file_list.remove(file)

    # @jkskolf, I don't think I like this ...
    for fname in file_list:
        from_file = os.path.join(data_input_path, fname)
        to_file = os.path.join(data_output_path, fname)
        print(from_file)
        shutil.copy(from_file, to_file)

clone_timeseries(input_directory, output_directory)

2022 v3 Annual Technology Baseline Workbook Mid-year update 2-15-2023 Clean.xlsx
2022 v3 Annual Technology Baseline Workbook Mid-year update 2-15-2023 Clean.xlsx
Bus_data_gen_weights_mappings.csv
candidate_generators_initial_list.csv
DAY_AHEAD_load.csv
DAY_AHEAD_solar.csv
DAY_AHEAD_wind.csv
ERCOT-Adjusted-Forecast.xlsb
ERCOT-Adjusted-Forecast.xlsb
new_gen_processing.ipynb
new_gen_processing.ipynb
REAL_TIME_solar.csv
REAL_TIME_wind.csv
reserves.csv
simulation_objects.csv
C:/Users/rundx/idaes-gtep/gtep/data/123_Bus_Coal\bus.csv
C:/Users/rundx/idaes-gtep/gtep/data/123_Bus_Coal\Bus_data_gen_weights_mappings.csv
C:/Users/rundx/idaes-gtep/gtep/data/123_Bus_Coal\candidate_generators_initial_list.csv
C:/Users/rundx/idaes-gtep/gtep/data/123_Bus_Coal\DAY_AHEAD_load.csv
C:/Users/rundx/idaes-gtep/gtep/data/123_Bus_Coal\DAY_AHEAD_solar.csv
C:/Users/rundx/idaes-gtep/gtep/data/123_Bus_Coal\DAY_AHEAD_wind.csv
C:/Users/rundx/idaes-gtep/gtep/data/123_Bus_Coal\network_inspection.ipynb
C:/Users/rundx/idae

In [68]:
adjusted_forecast = pd.read_excel("C:/Users/rundx/idaes-gtep/gtep/data/123_Bus_Coal/ERCOT-Adjusted-Forecast.xlsb")

In [ ]:
def update_load(input_data_path, output_data_path, adjusted_forecast):
    load_df = pd.read_csv(input_data_path + "/REAL_TIME_load.csv")
    adjusted_forecast = adjusted_forecast[adjusted_forecast["year"] == 2035]
    bus_df = pd.read_csv(input_data_path + "/bus.csv")
    bus_dict = dict(zip(bus_df["Bus ID"], bus_df["Zone"]))
    base_zones = ["base_economic_coast", "base_economic_east", "base_economic_fwest", "base_economic_ncent", "base_economic_north", "base_economic_scent", "base_economic_south", "base_economic_west"]
    scaled_zones = ["coast_net", "east_net", "fwest_net", "ncent_net", "north_net", "scent_net", "south_net", "west_net"]
    zones = ["coast", "east", "fwest", "ncent", "north", "scent", "south", "west"]
    cap_zones = [zone.upper() for zone in zones]
    for i,zone in enumerate(zones):
        adjusted_forecast["scaled_" + zone] = adjusted_forecast[scaled_zones[i]]/adjusted_forecast[base_zones[i]]
    column_list = ["year", "month", "day", "hour", "scaled_coast", "scaled_east", "scaled_fwest", "scaled_ncent", "scaled_north", "scaled_scent", "scaled_south", "scaled_west"]
    load_scaling_df = adjusted_forecast[column_list]
    scaled_names = ["scaled_" + zone for zone in zones]
    name_conversion_dict = dict(zip(scaled_names,cap_zones))
    load_scaling_df = load_scaling_df.rename(columns=name_conversion_dict)
    load_scaling_df = load_scaling_df.reset_index(drop=True)
    for k, v in bus_dict.items():
        load_scaling_df[k] = load_scaling_df[v]
    print(bus_dict)
    # print(load_scaling_df)
    print(load_df)
    scaled_load_df = load_df.mul(load_scaling_df, axis=0)
    print(scaled_load_df)
    scaled_load_df.to_csv(output_data_path + "/scaled_load.csv")
update_load(input_directory, output_directory, adjusted_forecast)

C:\Users\rundx\AppData\Local\Temp\ipykernel_13256\4072721463.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adjusted_forecast["scaled_" + zone] = adjusted_forecast[scaled_zones[i]]/adjusted_forecast[base_zones[i]]
C:\Users\rundx\AppData\Local\Temp\ipykernel_13256\4072721463.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adjusted_forecast["scaled_" + zone] = adjusted_forecast[scaled_zones[i]]/adjusted_forecast[base_zones[i]]
C:\Users\rundx\AppData\Local\Temp\ipykernel_13256\4072721463.py:11: Sett

{1: 'EAST', 2: 'NORTH', 3: 'FWEST', 4: 'COAST', 5: 'SOUTH', 6: 'NCENT', 7: 'NCENT', 8: 'SCENT', 9: 'EAST', 10: 'NCENT', 11: 'NCENT', 12: 'FWEST', 13: 'NCENT', 14: 'FWEST', 15: 'SCENT', 16: 'NORTH', 17: 'SCENT', 18: 'SOUTH', 19: 'SOUTH', 20: 'NCENT', 21: 'WEST', 22: 'WEST', 23: 'SCENT', 24: 'SCENT', 25: 'SOUTH', 26: 'FWEST', 27: 'NCENT', 28: 'WEST', 29: 'SCENT', 30: 'WEST', 31: 'SOUTH', 32: 'SOUTH', 33: 'SCENT', 34: 'WEST', 35: 'NORTH', 36: 'SCENT', 37: 'NCENT', 38: 'SOUTH', 39: 'COAST', 40: 'NCENT', 41: 'WEST', 42: 'NCENT', 43: 'SCENT', 44: 'EAST', 45: 'SCENT', 46: 'SOUTH', 47: 'SCENT', 48: 'WEST', 49: 'SOUTH', 50: 'FWEST', 51: 'SOUTH', 52: 'SCENT', 53: 'FWEST', 54: 'EAST', 55: 'NCENT', 56: 'SCENT', 57: 'COAST', 58: 'NORTH', 59: 'FWEST', 60: 'NCENT', 61: 'SCENT', 62: 'COAST', 63: 'NCENT', 64: 'SCENT', 65: 'SOUTH', 66: 'WEST', 67: 'COAST', 68: 'COAST', 69: 'COAST', 70: 'WEST', 71: 'SOUTH', 72: 'SOUTH', 73: 'SOUTH', 74: 'WEST', 75: 'NORTH', 76: 'NCENT', 77: 'WEST', 78: 'NCENT', 79: 'SCEN